# Investigações - Devoro
***

In [91]:
#data kit
import pandas as pd
import math
import numpy as np

#manipulacao de strings e regex
import re
from unidecode import unidecode

#google
from google.cloud import bigquery
from google.oauth2 import service_account

In [92]:
#!gcloud auth login

In [93]:
#Query para puxar os dados da ecommerce_sales - limpezas preliminares

client = bigquery.Client()

query = (
"""

select 

CASE
    WHEN client_name IS NULL OR client_name = '' OR client_name = 'RAPPI' THEN ''
  ELSE
  client_name
END
  AS client_name,
  CASE
    WHEN client_document IS NULL OR s.client_document = '' OR client_document IN('26900161000125', '17895646000691', '00000000000', '11111111111', '22222222222', '33333333333', '44444444444', '55555555555', '66666666666', '77777777777', '88888888888', '99999999999', '12345678909') or client_document = 'None' or length(client_document) < 11 THEN NULL 
    WHEN ((devoro.function.validador_1_cpf(client_document) = safe_cast(SUBSTR(client_document, 10,1) AS int64))
    OR (devoro.function.validador_1_cpf(client_document) = 10
    AND safe_cast(SUBSTR(client_document, 10,1) AS int64) = 0))
    AND (devoro.function.validador_2_cpf(client_document) = safe_cast(SUBSTR(client_document, 11,1) AS int64)
    OR (devoro.function.validador_2_cpf(client_document) = 10
    AND safe_cast(SUBSTR(client_document, 11,1) AS int64) = 0)) then client_document
    WHEN (devoro.function.validador_1_cnpj(client_document) < 2 and safe_cast(SUBSTR(client_document, 13,1) AS int64) = 0 
    or (abs(11 - devoro.function.validador_1_cnpj(client_document))) = safe_cast(SUBSTR(client_document, 13,1) AS int64))

    and (devoro.function.validador_2_cnpj(client_document) < 2 and safe_cast(SUBSTR(client_document, 14,1) AS int64) = 0 or 

    (abs(11 - devoro.function.validador_2_cnpj(client_document))) = safe_cast(SUBSTR(client_document, 14,1) AS int64))

    then client_document 
    ELSE
    null
    END
    AS client_document,
    client_document_type,
    CASE
    WHEN client_email IS NULL OR client_email = '' OR client_email IN ('rappi@go2go.com.br', 'integration.public.publicapi@rappi.com') THEN ''
  ELSE
  client_email
END
  AS client_email,
  CASE
    WHEN client_phone IS NULL OR client_phone = '' or length(replace(replace(replace(replace(replace(client_phone, '55',''), '+', ''), ')',''), "(", ''), "-", "")) < 11 OR s.client_phone LIKE "%0800%" OR s.client_phone LIKE '%49337360%' THEN NULL
   ELSE replace(replace(replace(replace(replace(client_phone, '55',''), '+', ''), ')',''), "(", ''), "-", "") 
  
END
  AS client_phone
from `devoro.sales.ecommerce_sales` s
where (client_email !='ClienteGo2GoTeste@gmail.com' or client_email is null) 

group by
1,2,3,4,5


union all 

  SELECT
         CONCAT(json_extract_scalar(customer,
        '$.firstName'), ' ', json_extract_scalar(customer,
        '$.lastName')) AS client_name,
         REPLACE(REPLACE(json_extract_scalar(customer,
          '$.cpf'), '.', ''), '-', '') AS client_document,
        'CPF' as client_document_type,
        json_extract_scalar(customer,
      '$.email') AS client_email,
         replace(replace(replace(replace(replace(json_extract_scalar(customer,
        '$.phone'), '55',''), '+', ''), ')',''), "(", ''), "-", "") as client_phone

  FROM
    `devoro.prod_tableUS.order_ecommerce` #emporio
    group by 1,2,3,4,5




""")




query_job_sales = client.query(query)

/usr/local/lib/python3.9/site-packages/google/auth/_default.py:79: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. We recommend you rerun `gcloud auth application-default login` and make sure a quota project is added. Or you can use service accounts instead. For more information about service accounts, see https://cloud.google.com/docs/authentication/
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)
/usr/local/lib/python3.9/site-packages/google/auth/_default.py:79: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. We recommend you rerun `gcloud auth application-default login` and make sure a quota project is added. Or you can use service accounts instead. For more information about service accounts, see https://clou

In [94]:
df_sale[df_sale['client_name'] == 'Wagner  Cozzolino']

,client_name,client_document,client_email,client_phone


In [95]:
df_sale = query_job_sales.to_dataframe()
df_sale

,client_name,client_document,client_document_type,client_email,client_phone
0,Nathalia Federmann,33359637844,CPF,,None
1,Rodrigo Borges,32262832862,CPF,,None
2,Raul Santos Neto,27736583889,CPF,,None
3,Beatriz Domingues,34996012811,CPF,,None
4,Ana,None,CNPJ,,None
...,...,...,...,...,...
195751,Andre Nascimento,07487748740,CPF,,None
195752,LUCIANO,29142240875,CPF,,None
195753,Natalia Lamounier dos Martires Guerra,26526751334,CPF,,None
195754,Denise Freire,14629931848,CPF,,None


In [96]:
#name
df_sale['client_name']=df_sale['client_name'].replace('', 'null', regex=True)
df_sale['client_name']=df_sale['client_name'].replace(r'^\s*$', 'null', regex=True)
df_sale['client_name']=df_sale['client_name'].replace('None', 'null', regex=True)
df_sale['client_name']=df_sale['client_name'].replace(r'\s+', ' ', regex=True)

#document
df_sale['client_document']=df_sale['client_document'].replace('', 'null', regex=True)
df_sale['client_document']=df_sale['client_document'].replace(r'^\s*$', 'null', regex=True)
df_sale['client_document']=df_sale['client_document'].replace('None', 'null', regex=True)
#email
df_sale['client_email']=df_sale['client_email'].str.lower()
df_sale['client_email']=df_sale['client_email'].replace(r'^\s*$', 'null', regex=True)
df_sale['client_email']=df_sale['client_email'].replace(r'^\s*$', 'null', regex=True)
df_sale['client_email']=df_sale['client_email'].replace('', 'null', regex=True)
df_sale['client_email']=df_sale['client_email'].replace('None', 'null', regex=True)
#phone
df_sale['client_phone']=df_sale['client_phone'].replace('', 'null', regex=True)
df_sale['client_phone']=df_sale['client_phone'].replace(r'^\s*$', 'null', regex=True)
#

df_sale = df_sale.fillna(value='null')

In [97]:
df_sale

,client_name,client_document,client_document_type,client_email,client_phone
0,Nathalia Federmann,33359637844,CPF,null,null
1,Rodrigo Borges,32262832862,CPF,null,null
2,Raul Santos Neto,27736583889,CPF,null,null
3,Beatriz Domingues,34996012811,CPF,null,null
4,Ana,null,CNPJ,null,null
...,...,...,...,...,...
195751,Andre Nascimento,07487748740,CPF,null,null
195752,LUCIANO,29142240875,CPF,null,null
195753,Natalia Lamounier dos Martires Guerra,26526751334,CPF,null,null
195754,Denise Freire,14629931848,CPF,null,null


In [98]:
client = bigquery.Client()

query = (
"""

SELECT * except(document_value), 
case when length(REGEXP_EXTRACT(replace(REPLACE(replace(REPLACE(document_value, '-', ''), '+1', ''), '.', ''), ' ', ''), r"[0-9]+")) = 11 
then document_value else null end as document_value
FROM EXTERNAL_QUERY("projects/devoro/locations/southamerica-east1/connections/conexao_dw_devoro", "SELECT * FROM wifi_guest;")



"""





) 




query_job_guests = client.query(query)

/usr/local/lib/python3.9/site-packages/google/auth/_default.py:79: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. We recommend you rerun `gcloud auth application-default login` and make sure a quota project is added. Or you can use service accounts instead. For more information about service accounts, see https://cloud.google.com/docs/authentication/
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)
/usr/local/lib/python3.9/site-packages/google/auth/_default.py:79: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. We recommend you rerun `gcloud auth application-default login` and make sure a quota project is added. Or you can use service accounts instead. For more information about service accounts, see https://clou

In [99]:
df_guest = query_job_guests.to_dataframe()
df_guest.head()

,id,last_access,status,email_is_valid,document_type,authorize_email,register_mode,zip_code,name,email,phone,created_at,updated_at,document_value
0,36910233,2021-05-15 23:47:49,ACTIVE,True,CPF,,Formulário,01529-020,Carolina,carolffsilva@gmail.com,27999496539,2021-05-15 23:47:46,2021-05-16 13:58:33.999276,12452732729
1,36910194,2021-05-15 23:42:38,ACTIVE,True,CPF,,Formulário,04678-002,Flávia Cristina Ferreira,flacrisf@gmail.com,11947468938,2021-05-15 23:42:35,2021-05-16 13:58:33.999276,28818935895
2,36910008,2021-05-15 23:21:49,ACTIVE,True,CPF,,Facebook,02722-060,Marcio Machado Gonçalves da Silveira,marciogood@hotmail.com,11991874756,2021-05-15 23:21:45,2021-05-16 13:58:33.999276,24806794899
3,36910005,2021-05-15 23:21:17,ACTIVE,True,CPF,,Formulário,09060-120,Eugenio Comparini,eugeniocomparini@gmail.com,11991935320,2021-05-15 23:21:13,2021-05-16 13:58:33.999276,41868499863
4,36909933,2021-05-15 23:13:22,ACTIVE,True,CPF,1,Facebook,06065-120,Vinicius Lima,vinicius.vielesfolado@hotmail.com,11993226093,2021-05-15 23:13:19,2021-05-16 13:58:33.999276,47261995878


## Definição de Funções

In [100]:
def clean_row(row):
    
    """ Remove caracteres especiais, mantendo letras e números """
    
    row = re.sub('[^A-Za-z\d ]+', '', row)
    return row

def transform_name(row):
    
    """ Separa e-mails que estão no campo 'name', troca pontos e traços por espaços e retira caracteres especiais """
    
    if row == 'nan':
        row = ''
    #remove acentuacao
    row = unidecode(row)
    #remove enderecos
    if bool(re.fullmatch('(Rua|R\.|r\.|R|rua|Avenida|Av\.|Av|av\.|avenida)[\w\s]+,*\s*\d+[\w\s-]*', row)):
        row = ''
    #separa e-mail
    if bool(re.fullmatch('[\w\._-]+@\w+(\.\w+)+', row)):
        row = row.split('@')[0]
    #tira numeros
    row = re.sub('\d', '', row)
    #troca pontos e tracos por espacos
    row = row.replace('.', ' ').replace('_', ' ').replace('-', ' ')
    #tira caracteres especiais
    row = re.sub('[^A-Za-z ]+', '', row)
    #limpa a string caso ela seja composta somente de espacos
    if bool(re.fullmatch('\s+', row)):
        row = ''
    #deixa todas as letras minusculas
    row = row.lower()
    return str.title(row).strip()

def fill_name(name, email):
    
    """ Preenche nomes vazios com a primeira parte do e-mail
    Uso: df['name'] = df.apply(lambda x: preenche_nome(x['name'], x['email']), axis = 1)"""
    
    global count_email
    
    if name == '':
        name = email.split('@')[0]
        #tira numeros
        name = re.sub('\d', '', name)
        #troca pontos e tracos por espacos
        name = name.replace('.', ' ').replace('_', ' ').replace('-', ' ')
        if name != '' and pd.notnull(name):
            count_email += 1
        return str.title(name)
    else:
        return str.title(name)

def compare_names(name_guest, name_sale):
    
    """ Escolhe o nome mais adequado para o dataframe definitivo, comparando o tamanho e a intersecção entre os 2 nomes disponíveis """
    
    global count_name_guest
    
    if(pd.isnull(name_sale) or name_sale == ''):
        return name_guest
    else:
        name_guest = unidecode(name_guest).lower().strip()
        name_sale = unidecode(name_sale).lower().strip()

        name_guest_list = name_guest.split(' ')
        name_sale_list = name_sale.split(' ')

        if (len(name_sale_list) == 1 and len(name_guest_list) > len(name_sale_list) and bool(list(set(name_guest_list).intersection(name_sale_list)))):
            count_name_guest += 1
            return str.title(name_guest)

        elif (len(name_sale_list) > 1 and len(name_guest_list) > len(name_sale_list) and len(list(set(name_guest_list).intersection(name_sale_list))) > 1):
            count_name_guest += 1
            return str.title(name_guest)

        else:
            return str.title(name_sale)
    
def complete_name(client_name, email):
    
    """ Completa nomes a partir do e-mail. Os nomes adicionados ficam entre () para indicar sua reconstrução """ 
    
    global count_email
    
    names = client_name.lower().split(' ')
    email_user = email.split('@')[0]
    email_user = re.sub('\d', '', email_user)
    email_user = email_user.replace('.', ' ').replace('_', ' ').replace('-', ' ')
    intersec_exists = False
    for n in names:
        if n in email_user:
            email_user = email_user.replace(n, '')
            intersec_exists = True
    if intersec_exists and email_user.strip() != '':
        count_email += 1
        return str.title(re.sub('\s+',' ',client_name + ' (' + email_user.strip() +')'))
    else:
        return ''

***

## Ingestão de Dados - Guests

In [101]:
#df_guest = pd.read_csv('../datasets/devoro_guest.csv')
shape_guest = df_guest.shape

## Aplicação do ETL - Guests

> <b>Tratamentos Realizados no Campo 'name':</b> <br>
Limpeza -> remoção de caracteres especiais e endereços e, caso haja um e-mail no lugar do nome, uso do 'login' <br>
Preenchimento -> preenchimento de nomes vazios usando o 'login' do e-mail

In [102]:
count_email = 0
df_guest['email'] = df_guest['email'].str.lower()
#remocao de caracteres especiais do cpf e do zip-code
#df_guest.loc[:, 'document_value'] = df_guest.loc[:,'document_value'].apply(clean_row)
df_guest.loc[pd.notnull(df_guest['zip_code']), 'zip_code'] = df_guest.loc[pd.notnull(df_guest['zip_code']),'zip_code'].apply(clean_row)
#passaportes com 11 digitos -> CPF
#df_guest['document_type'] = df_guest.apply(lambda x: 'CPF' if len(x['document_value']) == 11 and str.isdecimal(x['document_value']) else x['document_type'] ,axis = 1)
#cast da coluna name p/ string
df_guest.loc[:, 'name'] = df_guest.loc[:,'name'].astype(str)
#tratamento da coluna name
df_guest.loc[df_guest['document_type'] == 'CPF', 'name'] = df_guest.loc[df_guest['document_type'] == 'CPF','name'].apply(transform_name)
#preenchimento de valores vazios com login vindo do e-mail
df_guest.loc[df_guest['document_type'] == 'CPF','name'] = df_guest.loc[df_guest['document_type'] == 'CPF'].apply(lambda x: fill_name(x['name'], x['email']), axis = 1)

In [103]:
df_guest.head()

,id,last_access,status,email_is_valid,document_type,authorize_email,register_mode,zip_code,name,email,phone,created_at,updated_at,document_value
0,36910233,2021-05-15 23:47:49,ACTIVE,True,CPF,,Formulário,01529020,Carolina,carolffsilva@gmail.com,27999496539,2021-05-15 23:47:46,2021-05-16 13:58:33.999276,12452732729
1,36910194,2021-05-15 23:42:38,ACTIVE,True,CPF,,Formulário,04678002,Flavia Cristina Ferreira,flacrisf@gmail.com,11947468938,2021-05-15 23:42:35,2021-05-16 13:58:33.999276,28818935895
2,36910008,2021-05-15 23:21:49,ACTIVE,True,CPF,,Facebook,02722060,Marcio Machado Goncalves Da Silveira,marciogood@hotmail.com,11991874756,2021-05-15 23:21:45,2021-05-16 13:58:33.999276,24806794899
3,36910005,2021-05-15 23:21:17,ACTIVE,True,CPF,,Formulário,09060120,Eugenio Comparini,eugeniocomparini@gmail.com,11991935320,2021-05-15 23:21:13,2021-05-16 13:58:33.999276,41868499863
4,36909933,2021-05-15 23:13:22,ACTIVE,True,CPF,1,Facebook,06065120,Vinicius Lima,vinicius.vielesfolado@hotmail.com,11993226093,2021-05-15 23:13:19,2021-05-16 13:58:33.999276,47261995878


***

## Ingestão de Dados - Sale

In [104]:
#df_sale = pd.read_csv('../datasets/devoro_sale.csv')
shape_sale = df_sale.shape

In [105]:


#df_sale[df_sale['client_email'] == 'fabiodantas@ciatc.com.br']


## Aplicação do ETL - Sales

> <b>Tratamentos Realizados no Campo 'client_name':</b> <br>
Limpeza -> remoção de caracteres especiais e endereços e, caso haja um e-mail no lugar do nome, uso do 'login' <br>
Preenchimento -> preenchimento de nomes usando a coluna 'name' do dataframe 'guests'

In [106]:
#tratamento da coluna client_phone - remocao de espacos e traços entre os numeros
df_sale['client_phone'] = df_sale['client_phone'].str.lstrip("0")
df_sale['client_phone'] = df_sale['client_phone'].str.replace(" ", "").replace("  ", "").replace('-', '')


#tratamento da coluna client_name e remocao de caracteres especiais da coluna client_document
df_sale.loc[:, 'client_email'] = df_sale.loc[:,'client_email'].astype(str)
df_sale.loc[:, 'client_document'] = df_sale.loc[:,'client_document'].astype(str)
df_sale.loc[:, 'client_document'] = df_sale.loc[:, 'client_document'].apply(clean_row)
df_sale.loc[:, 'client_name'] = df_sale.loc[:,'client_name'].astype(str)
df_sale.loc[:, 'client_name'] = df_sale.loc[:,'client_name'].apply(transform_name)
df_sale.loc[df_sale['client_document_type'] == 'CPF', 'client_name'] = df_sale.loc[df_sale['client_document_type'] == 'CPF'].apply(lambda x: fill_name(x['client_name'], x['client_email']), axis = 1)


#copia das colunas de nome    
df_name_guest = df_guest.loc[(
             pd.notnull(df_guest['document_value']))
             & (df_guest['register_mode'] != 'Instagram')
             & (df_guest['register_mode'] != 'Twitter'),['name', 'document_value']].copy()
df_name_sale = df_sale.loc[df_sale['client_document'].notnull() ,['client_name', 'client_document']].copy()


#merge dos 2 dataframes
# df_merge = pd.merge(df_name_guest, df_name_sale, how = 'inner', left_on = 'document_value', right_on = 'client_document')
# shape_merge = df_merge.shape
# #cast das colunas de nome p/ string
# df_merge.loc[:, 'client_name'] = df_merge.loc[:,'client_name'].astype(str)
# df_merge.loc[:, 'name'] = df_merge.loc[:,'name'].astype(str)
# count_name_guest = 0
# df_merge.loc[:,'def_name'] = df_merge.apply(lambda x: compare_names(x['name'], x['client_name']), axis = 1)
# #drop de duplicatas e colunas desnecessarias
# df_merge = df_merge.drop(['name', 'document_value', 'client_name'], axis=1)
# df_merge = df_merge.drop_duplicates(subset=['client_document'])
# shape_merge_final = df_merge.shape

In [107]:
df_sale = df_sale[['client_name', 'client_document', 'client_email', 'client_phone']]
df_sale.head()

,client_name,client_document,client_email,client_phone
0,Nathalia Federmann,33359637844,null,null
1,Rodrigo Borges,32262832862,null,null
2,Raul Santos Neto,27736583889,null,null
3,Beatriz Domingues,34996012811,null,null
4,Ana,null,null,null


In [108]:
#merge com a sale original p/ gerar dataframe sale final
# df_sale_final = pd.merge(df_sale, df_merge, how = 'left', on = 'client_document')
# df_sale_final.loc[:,'client_name'] = df_sale_final.loc[:,'client_name'].astype(str)
# df_sale_final.loc[:,'def_name'] = df_sale_final.loc[:,'def_name'].astype(str)
# df_sale_final.loc[:,'client_name'] = df_sale_final.apply(lambda x: x['def_name'] if (x['def_name'] != '' and x['def_name'] != 'nan') else x['client_name'], axis = 1)
# #df_sale_final = df_sale_final.drop(['def_name', 'products'], axis = 1)
# #df_sale_final['date'] = pd.to_datetime(df_sale_final['date'])
# df_sale_final.loc[df_sale_final['client_name'] == 'Nan', 'client_name'] = math.nan
# df_sale_final.loc[df_sale_final['client_document'] == 'nan', 'client_document'] = math.nan
# shape_sale_final = df_sale_final.shape

In [109]:
# df_guest_final = pd.merge(df_guest, df_sale_final.loc[df_sale_final['client_document'].notnull(), ['client_name', 'client_document']].drop_duplicates(),
#                     how='left', left_on='document_value', right_on='client_document')
# df_guest_final.loc[:,'name'] = df_guest_final.apply(lambda x: x['client_name'] if (x['client_name'] != '' and pd.notnull(x['client_name'])) else x['name'], axis = 1)
# df_guest_final = df_guest_final.drop(['client_name', 'client_document'], axis=1)
# #preenchimento de sobrenomes usando e-mail
# df_guest_final['nome_preenchido_email'] = df_guest_final.apply(lambda x: complete_name(x['name'], x['email']), axis=1)
# shape_guest_final = df_guest_final.shape

In [110]:
#acompanhamento do tamanho dos datasets e de estatisticas
# print('shape sale:  inicial: ' + str(shape_sale) + '  final: ' + str(shape_sale_final))
# print('shape guests:  inicial: ' + str(shape_guest) + '  final: ' + str(shape_guest_final))
# print('shape merge:  inicial: ' + str(shape_merge) + '  final: ' + str(shape_merge_final))
# print('% de nomes atualizados com login de email: ' + str(round(100 * count_email/len(df_guest), 2)) + '%')
# print('% de CPFs de go2go.sale atualizados com nomes de guests: ' + str(round(100 * count_name_guest/len(df_sale), 2)) + '%')

In [111]:
# Construct a BigQuery client object.
client = bigquery.Client()

# TODO(developer): Set table_id to the ID of the table to fetch.
table_id = 'devoro.golden_record.aux_sale'

# If the table does not exist, delete_table raises
# google.api_core.exceptions.NotFound unless not_found_ok is True.
client.delete_table(table_id, not_found_ok=True)  # Make an API request.
print("Deleted table '{}'.".format(table_id))

/usr/local/lib/python3.9/site-packages/google/auth/_default.py:79: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. We recommend you rerun `gcloud auth application-default login` and make sure a quota project is added. Or you can use service accounts instead. For more information about service accounts, see https://cloud.google.com/docs/authentication/
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)
/usr/local/lib/python3.9/site-packages/google/auth/_default.py:79: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. We recommend you rerun `gcloud auth application-default login` and make sure a quota project is added. Or you can use service accounts instead. For more information about service accounts, see https://clou

Deleted table 'devoro.golden_record.aux_sale'.


In [112]:
# Construct a BigQuery client object.
client = bigquery.Client()

# TODO(developer): Set table_id to the ID of the table to fetch.
table_id = 'devoro.golden_record.aux_guest'

# If the table does not exist, delete_table raises
# google.api_core.exceptions.NotFound unless not_found_ok is True.
client.delete_table(table_id, not_found_ok=True)  # Make an API request.
print("Deleted table '{}'.".format(table_id))

/usr/local/lib/python3.9/site-packages/google/auth/_default.py:79: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. We recommend you rerun `gcloud auth application-default login` and make sure a quota project is added. Or you can use service accounts instead. For more information about service accounts, see https://cloud.google.com/docs/authentication/
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)
/usr/local/lib/python3.9/site-packages/google/auth/_default.py:79: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. We recommend you rerun `gcloud auth application-default login` and make sure a quota project is added. Or you can use service accounts instead. For more information about service accounts, see https://clou

Deleted table 'devoro.golden_record.aux_guest'.


In [113]:
#Cria a tabela aux_sale

table_id = "devoro.golden_record.aux_sale"




job_config = bigquery.LoadJobConfig(
    schema = [
        bigquery.SchemaField("client_name", "STRING", mode="NULLABLE"),
        bigquery.SchemaField("client_document", "STRING", mode="NULLABLE"),
#        bigquery.SchemaField("client_document_type", "STRING", mode="NULLABLE"),
        bigquery.SchemaField("client_email ", "STRING", mode="NULLABLE"),
         bigquery.SchemaField("client_phone", "STRING", mode="NULLABLE")
    ],write_disposition="WRITE_TRUNCATE")


job = client.load_table_from_dataframe(df_sale, table_id)




In [114]:
#Cria a tabela aux_guest

table_id = "devoro.golden_record.aux_guest"

# job_config = bigquery.LoadJobConfig(
#     schema = [
#         bigquery.SchemaField("id", "STRING", mode="REQUIRED"),
#         bigquery.SchemaField("documentType", "STRING", mode="REQUIRED"),
# #        bigquery.SchemaField("documentType", "STRING", mode="REQUIRED"),
#         bigquery.SchemaField("email", "STRING", mode="REQUIRED"),
#     ],write_disposition="WRITE_TRUNCATE")


job = client.load_table_from_dataframe(df_guest, table_id)

In [115]:
#Criando tabela aux_sale transformando ''null'' para null 

client = bigquery.Client()

query = (
"""
create or replace table `devoro.golden_record.aux_sale` as ( 
select
case when lower(client_name) = 'null' or client_name = '' then null else client_name end as client_name,
case when client_document = 'null' or client_document = '' then null else client_document end as client_document,
case when client_email = 'null' or client_email = '' then null else client_email end as client_email, 
case when client_phone = 'null' or client_phone = '' or client_phone ='11999999999' then null else client_phone end as client_phone
from 
`devoro.golden_record.aux_sale`
)""")




query_job_sales = client.query(query)
query_job_sales

/usr/local/lib/python3.9/site-packages/google/auth/_default.py:79: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. We recommend you rerun `gcloud auth application-default login` and make sure a quota project is added. Or you can use service accounts instead. For more information about service accounts, see https://cloud.google.com/docs/authentication/
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)
/usr/local/lib/python3.9/site-packages/google/auth/_default.py:79: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. We recommend you rerun `gcloud auth application-default login` and make sure a quota project is added. Or you can use service accounts instead. For more information about service accounts, see https://clou

QueryJob<project=devoro, location=US, id=d8a2e3f5-594a-4962-bad8-67d1875b8b83>

In [116]:
client = bigquery.Client()

query = (
"""

create or replace table `devoro.golden_record.aux_guest` as ( 
select 

* except(document_value), 
case when ((devoro.function.validador_1_cpf(document_value) = safe_cast(SUBSTR(document_value, 10,1) AS int64))
OR (devoro.function.validador_1_cpf(document_value) = 10
  AND safe_cast(SUBSTR(document_value, 10,1) AS int64) = 0))
AND (devoro.function.validador_2_cpf(document_value) = safe_cast(SUBSTR(document_value, 11,1) AS int64)
OR (devoro.function.validador_2_cpf(document_value) = 10
  AND safe_cast(SUBSTR(document_value, 11,1) AS int64) = 0)) then document_value else null end as document_value



from (



select 
* except(document_value, name, email, phone), case when lower(name) = 'null' or name = '' then null else name end as name,
case when document_value = 'null' or document_value = '12345678909' or document_value = '00000000191' or document_value = '' then null else document_value end as document_value,
case when email = 'null' or email = '' then null else email end as email, 
case when phone = 'null' or phone = '' or phone = '11999999999' or length(phone) < 11 or phone = '1199999999' then null else phone end as phone




from `devoro.golden_record.aux_guest`)
)
""")

query_job_guest = client.query(query)
query_job_guest

/usr/local/lib/python3.9/site-packages/google/auth/_default.py:79: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. We recommend you rerun `gcloud auth application-default login` and make sure a quota project is added. Or you can use service accounts instead. For more information about service accounts, see https://cloud.google.com/docs/authentication/
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)
/usr/local/lib/python3.9/site-packages/google/auth/_default.py:79: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. We recommend you rerun `gcloud auth application-default login` and make sure a quota project is added. Or you can use service accounts instead. For more information about service accounts, see https://clou

QueryJob<project=devoro, location=US, id=bcfc00b3-b753-4a22-ac02-4e8392c7432d>

In [117]:
client = bigquery.Client()

query = (
"""



create or replace table `devoro.golden_record.cadastro_cpf` as ( 
select 

*
from `devoro.function.aux_transaction`()
)
""")

query_job_sales = client.query(query)
client.query(query)

/usr/local/lib/python3.9/site-packages/google/auth/_default.py:79: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. We recommend you rerun `gcloud auth application-default login` and make sure a quota project is added. Or you can use service accounts instead. For more information about service accounts, see https://cloud.google.com/docs/authentication/
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)
/usr/local/lib/python3.9/site-packages/google/auth/_default.py:79: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. We recommend you rerun `gcloud auth application-default login` and make sure a quota project is added. Or you can use service accounts instead. For more information about service accounts, see https://clou

QueryJob<project=devoro, location=US, id=d9e0d596-4e6d-4960-92ce-e0bf85ff19fc>

In [118]:
# #Recriacao da cadastro_cpf, incluindo agora dados do Empório


# client = bigquery.Client()

# query = (
# """



# create or replace table `devoro.golden_record.cadastro_cpf` as ( 



# with client_document as ( 
# select
# s.client_document, 
# if(s.client_name is null or s.client_name = '', d.name, s.client_name) as client_name, 
# if(s.client_email is null or s.client_email = '', d.email, s.client_email) as client_email, 
# if(s.client_phone is null or s.client_phone = '', d.phone, s.client_phone) as client_phone
# from `devoro.golden_record.cadastro_cpf` s 

# left join `devoro.function.aux_emporio`() d
# on s.client_document = d.document
# and s.client_document !=''


# group by 1,2,3,4
# ),
# client_phone as ( 
# select
# if(s.client_name is null or s.client_name = '', d.name, s.client_name) as client_name, 
# if(s.client_document is null or s.client_document = '', d.document, s.client_document) as client_document, 
# if(s.client_email is null or s.client_email = '', d.email, s.client_email) as client_email, 
# s.client_phone
# from client_document s 

# left join `devoro.function.aux_emporio`() d
# on s.client_phone = d.phone 
# and s.client_phone !=''
# group by 1,2,3,4
# )
# select
# if(s.client_name is null or s.client_name = '', d.name, s.client_name) as client_name, 
# if(s.client_document is null or s.client_document = '', d.document, s.client_document) as client_document, 
# s.client_email,
# if(s.client_phone is null or s.client_phone = '', d.phone, s.client_phone) as client_phone
# from client_phone  s 

# left join `devoro.function.aux_emporio`() d
# on s.client_email = d.email
# and s.client_email != ''

# group by 1,2,3,4
# """)

# query_job_sales = client.query(query)
# client.query(query)


In [119]:
# #Trazer dados de empório para a cadastro cpf, ou seja, 
# #aqueles que nao existem ainda na cadastro cpf mas estao na emporio

# client = bigquery.Client()

# query = (
# """



# create or replace table `devoro.golden_record.cadastro_cpf` as ( 



# select
# client_document, client_name, client_email, client_phone
# from `devoro.golden_record.cadastro_cpf` c

# union all

# select
# document, name, email, phone
# from `devoro.function.aux_emporio`() c
# where document in (
# select
# distinct(document)
# from `devoro.function.aux_emporio`() c
# except distinct(

#     select
#     client_document
#     from `devoro.golden_record.cadastro_cpf`
#     group by 1
# ))

# """)

# query_job_sales = client.query(query)
# client.query(query)


In [120]:
client = bigquery.Client()

query = (
"""

create or replace table `devoro.golden_record.transaction` as ( 
with ecommerce_sales as ( 
select 
case when extract(date from datetime(date, "America/Sao_Paulo")) in ('2021-11-02', '2021-11-04') then extract(date from datetime(date, "America/Sao_Paulo")) else 
datetime(date, "America/Sao_Paulo") end as date,
case when marketplace_sale_id in ('App-441491-App-441491', 'App-19830227-App-19830227', 'App-3247469-App-3247469', 'App-18361162-App-18361162', 'App-13559089-App-13559089', '2692-7023527984996077') then marketplace_sale_id else concat(id, marketplace_sale_id) end as id_pedido, client_document_type, origin, true as isDelivery, total_value,
CASE
    WHEN client_name IS NULL OR client_name = '' OR client_name = 'RAPPI' THEN NULL
  ELSE
  client_name
END
  AS client_name,
 CASE
    WHEN client_document IS NULL OR s.client_document = '' OR client_document IN('26900161000125', '17895646000691', '00000000000', '11111111111', '22222222222', '33333333333', '44444444444', '55555555555', '66666666666', '77777777777', '88888888888', '99999999999', '12345678909') or client_document = 'None' or length(client_document) < 11 THEN NULL 
    when ((devoro.function.validador_1_cpf(client_document) = safe_cast(SUBSTR(client_document, 10,1) AS int64))
OR (devoro.function.validador_1_cpf(client_document) = 10
  AND safe_cast(SUBSTR(client_document, 10,1) AS int64) = 0))
AND (devoro.function.validador_2_cpf(client_document) = safe_cast(SUBSTR(client_document, 11,1) AS int64)
OR (devoro.function.validador_2_cpf(client_document) = 10
  AND safe_cast(SUBSTR(client_document, 11,1) AS int64) = 0)) then client_document

    WHEN (devoro.function.validador_1_cnpj(client_document) < 2 and safe_cast(SUBSTR(client_document, 13,1) AS int64) = 0 
    or (abs(11 - devoro.function.validador_1_cnpj(client_document))) = safe_cast(SUBSTR(client_document, 13,1) AS int64))

    and (devoro.function.validador_2_cnpj(client_document) < 2 and safe_cast(SUBSTR(client_document, 14,1) AS int64) = 0 or 

    (abs(11 - devoro.function.validador_2_cnpj(client_document))) = safe_cast(SUBSTR(client_document, 14,1) AS int64))

    then client_document 
    ELSE
    null
    END
  AS client_document,
  CASE
    WHEN client_email IS NULL OR client_email = '' OR client_email IN ('rappi@go2go.com.br', 'integration.public.publicapi@rappi.com') THEN NULL
  ELSE
  client_email
END
  AS client_email,
  CASE
    WHEN client_phone IS NULL OR client_phone = '' OR s.client_phone LIKE "%0800%" OR length(replace(replace(replace(replace(replace(s.client_phone, '55',''), '+', ''), ')',''), "(", ''), "-", "")) < 11 or s.client_phone LIKE '%49337360%' THEN NULL
   ELSE replace(replace(replace(replace(client_phone, '55',''), '+', ''), ')',''), "(", '')
  
END
  AS client_phone
from `devoro.sales.ecommerce_sales` s
where (client_email !='ClienteGo2GoTeste@gmail.com' or client_email is null) and origin != 'CENTRAL' 
group by 1,2,3,4,5,6,7,8,9,10
),


client_document as ( 
select 
date, id_pedido, client_document_type, t.client_document, coalesce(t.client_email, f.client_email) as client_email, coalesce(t.client_phone, f.client_phone) as client_phone,origin, isDelivery, total_value, 
 from ecommerce_sales t
left join `devoro.golden_record.cadastro_cpf` f
on (f.client_document = t.client_document) 
group by 1,2,3,4,5,6,7,8,9
),



client_email as ( 
select 
date, id_pedido, client_document_type, coalesce(t.client_document, f.client_document) as client_document, coalesce(t.client_phone, f.client_phone) as client_phone, origin, isDelivery, total_value, f.client_email, 
 from client_document t
left join `devoro.golden_record.cadastro_cpf` f
on (f.client_email = t.client_email) 
group by 1,2,3,4,5,6,7,8,9 
)



select 
date, id_pedido, client_document_type,# case when t.client_document is null and f.client_document is null then t.client_email end as client_document


case 
when t.client_document is null and f.client_document is null and t.client_email is null and f.client_email is null then cast(coalesce(to_hex(sha256(t.client_phone)), to_hex(sha256(f.client_phone))) as string)
when t.client_document is null and f.client_document is null and t.client_phone is null and f.client_phone is null then coalesce(to_hex(sha256(t.client_email)), to_hex(sha256(f.client_email)))
else coalesce(t.client_document, f.client_document) end as client_document
, origin, isDelivery, total_value
from client_email t
left join `devoro.golden_record.cadastro_cpf` f
ON
(f.client_phone = t.client_phone)
group by 1,2,3,4,5,6,7

union all

select 
time as date, concat(time, code_nf, store_cnpj) as id_pedido, document_type, document_value as client_document, 'nf' as origin, false as isDelivery, total_value

from `devoro.sales.ciatc_nfe`
where isDelivery is false
group by 1,2,3,4,5,6,7

union all

SELECT
  datetime(created_at) AS date,
  cast(id as string) AS id_pedido,
  'CPF' AS document_type,
 REPLACE(REPLACE(json_extract_scalar(customer,
          '$.cpf'), '.', ''), '-', '') AS document,
  'emporio' AS origin,
  TRUE AS isdelivery,
  total_cents/100 AS total_value,
  
FROM
  `devoro.prod_tableUS.order_ecommerce`
)

""")

query_job_sales = client.query(query)
client.query(query)



/usr/local/lib/python3.9/site-packages/google/auth/_default.py:79: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. We recommend you rerun `gcloud auth application-default login` and make sure a quota project is added. Or you can use service accounts instead. For more information about service accounts, see https://cloud.google.com/docs/authentication/
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)
/usr/local/lib/python3.9/site-packages/google/auth/_default.py:79: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. We recommend you rerun `gcloud auth application-default login` and make sure a quota project is added. Or you can use service accounts instead. For more information about service accounts, see https://clou

QueryJob<project=devoro, location=US, id=aa51014d-3174-46ad-94e9-b2044a0a4b24>

In [121]:
#Criação da user_segmentation 
client = bigquery.Client()

query = (
"""
create or replace table `devoro.golden_record.user_segmentation` as ( 
SELECT
  client_document,
  client_document_type,
  DATE_DIFF(CURRENT_DATE(), MAX(EXTRACT(date
      FROM
        date)), day) AS recencia,
  ROUND(SUM(total_value),2) AS ltv,
  ROUND(SUM(total_value)/COUNT(DISTINCT(id_pedido)),2) AS ticket_medio,
  COUNT(DISTINCT(date)) AS frequencia_data,
  COUNT(DISTINCT(id_pedido)) AS numero_pedidos,
  NULL AS cluster,
  NULL AS tag
FROM
  `devoro.golden_record.transaction`
WHERE
  client_document IS NOT NULL
  AND date BETWEEN DATE_SUB(CURRENT_DATE(), INTERVAL 12 month)
  AND CURRENT_DATE()
GROUP BY
  1,
  2)
  """)
query_job_sales = client.query(query)


/usr/local/lib/python3.9/site-packages/google/auth/_default.py:79: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. We recommend you rerun `gcloud auth application-default login` and make sure a quota project is added. Or you can use service accounts instead. For more information about service accounts, see https://cloud.google.com/docs/authentication/
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)
/usr/local/lib/python3.9/site-packages/google/auth/_default.py:79: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. We recommend you rerun `gcloud auth application-default login` and make sure a quota project is added. Or you can use service accounts instead. For more information about service accounts, see https://clou

***